## バックテスト用コード

In [ ]:
flg = {
    'buy_signal': 0, # False
    'sell_signal': 0, # False
    'order': {
        'exist': False,
        'side': '',
        'price': 0,
        'count': 0,
    },
    'position':{
        'exist': False,
        'side': '',
        'price': 0,
        'count': 0,
    },
    'records': {
        'buy_count': 0,
        'buy_winning': 0,
        'buy_return': [], # 利益率
        'buy_profit': [],
        'buy_holding_periods': [],

        'sell_count': 0,
        'sell_winning': 0,
        'sell_return': [], # 利益率
        'sell_profit': [],
        'sell_holding_periods': [],

        'slippage': [],
        'log': [],         # textに書き出すログ内容
    }

}

In [2]:
import requests
from datetime import datetime
import time
import json
import ccxt
import numpy as np

# テスト用の初期設定
chart_sec = 300 # 5分足
lot = 1 # トレードの枚数
slippage = 0.0005 #手数料やスリッページ
close_condition = 0 #n足後経過するまでは決済しない

# データ取得
def get_price(min, before = 0, after = 0):
    price = []
    params = {'periods', min}
    if before:
        params['before'] = before
    if after:
        params['after'] = after

    response = requests.get('https://api.cryptowat.ch/markets/bitflyer/btcfxjpy/ohlc', params)
    data = response.json()

    if data['result'][str(min)]:
        for i in data['result'][str(min)]:
            if i[1] and i[2] and i[3] and i[4]:
                price.append({
                    'close_time': i[0],
                    'close_time_dt': datetime.fromtimestamp(i[0]).strftime('%Y/%m/%d %H:%M'),
                    'open_price': i[1],
                    'high_price': i[2],
                    'low_price': i[3],
                    'close_price': i[4],
                })
        return price
    else:
        print('データがありません')
        return

# jsonファイルからの読み込み
def get_price_from_file(path):
    file = open(path, 'r', encoding='UTF-8')
    price = json.load(file)
    return price

# データと時間の表示
def print_price(data):
    print('時間: ' + datetime.fromtimestamp(data['close_time']).strftime('%Y/%m/%d %H:%M') + '始値: ' + str(data['open_price']) + '終値: ' + str(data['close_price']))

# データと時間をロギング
def log_price(data):
    log = '時間: ' + datetime.fromtimestamp(data['close_time']).strftime('%Y/%m/%d %H:%M') + '始値: ' + str(data['open_price']) + '終値: ' + str(data['close_price']) + '\n'
    flg['records']['log'].append(log) #ログの内容もflgに追記していいく
    return flg #　ログが追記された　フラグを返す

# candleが陽線、陰線の基準を満たしているかどうか
def check_candle(data, side):
    try:
        realbody_rate = abs(data['close_price'] - data['open_price'] / data['high_price'] - data['low_price'])
        increase_rate = data['close_price'] / data['open_price'] - 1
    except ZeroDivisionError as e:
        return False

    if side == 'buy':
        if data['close_price'] < data['open_price']:
            return False
        # elif increase_rate < 0.0003:
        #     return False
        # elif realbody_rate < 0.5:
        #     return False
        else:
            return True

    if side == 'sell':
        if data['close_price'] > data['open_price']:
            return False
        elif increase_rate > -0.0003:
            return False
        elif realbody_rate < 0.5:
            return False
        else:
            return True

# candleが連続上昇してるか
def check_ascend(data, last_data):
    if data['open_price'] > last_data['open_price'] and data['close_price'] > last_data['close_price']:
		return True
	else:
		return False

# candleが連続下落してるか
def check_descen(data, last_data):
	if data['open_price'] < last_data['open_price'] and data['close_price'] < last_data['close_price']:
		return True
	else:
		return False

# buy_signalで指値を出す
def buy_signal( data,last_data,flg ):
	if flg['buy_signal'] == 0 and check_candle( data,'buy' ):
		flg['buy_signal'] = 1
	elif flg['buy_signal'] == 1 and check_candle( data,'buy' )  and check_ascend( data,last_data ):
		flg['buy_signal'] = 2
	elif flg['buy_signal'] == 2 and check_candle( data,'buy' )  and check_ascend( data,last_data ):
		log = '３本連続で陽線 なので' + str(data['close_price']) + '円で買い指値を入れます\n'
		flg['records']['log'].append(log)
		flg['buy_signal'] = 3
		
		# ここにapiのbuyのコード

		flg['order']['exist'] = True
		flg['order']['side'] = 'BUY'
		flg['order']['price'] = round(data['close_price'] * lot)
	else:
		flg['buy_signal'] = 0
	return flg

# sell_signalで指値を出す
def sell_signal( data, last_data, flg ):
	if flg['sell_signal'] == 0 and check_candle( data,'sell' ):
		flg['sell_signal'] = 1

	elif flg['sell_signal'] == 1 and check_candle( data,'sell' )  and check_descend( data,last_data ):
		flg['sell_signal'] = 2

	elif flg['sell_signal'] == 2 and check_candle( data,'sell' )  and check_descend( data,last_data ):
		log = '３本連続で陰線 なので' + str(data['close_price']) + '円で売り指値を入れます\n'
		flg['records']['log'].append(log)
		flg['sell_signal'] = 3
		
		# ここにapiのsellコードを入れる

		flg['order']['exist'] = True
		flg['order']['side'] = 'SELL'
		flg['order']['price'] = round(data['close_price'] * lot)
		
	else:
		flg['sell_signal'] = 0
	return flg

    # 手仕舞いsignalで成行決済
    # 足ごとにポジションがあればメインコードから実行される。ポジションがある時の足の本数分だけ実行されることになる
    def close_position(data, last_data, flg):
        flg['position']['count'] += 1    # ポジションを持っている足のチェックごとにインクリメント。ポジションがない足の時はカウントしない。決済されたら、0にリセット

        if flg['position']['side'] == 'BUY':
            if data['close_price'] < last_data['close_price'] and flg['position']['count'] > close_condition:
                log = '前回の終値を下回ったので' + str(data['close_price']) + '円あたりで成行で決済します\n'
                flg['records']['log'].append(log)

                # 成行決済のコード

                records(flg, data)
                flg['position']['exist'] = False
                flg['position']['count'] = 0

        if flg['position']['side'] == 'SELL':
            if data['close_price'] > last_data['close_peice'] and flg['position']['count'] > close_condition:
                log = '前回の終値を上回ったので' + str(data['close_price']) + '円あたりで成行で決済します\n'
                flg['records']['log'].append(log)
                
                # 成行決済のコード

                records( flg,data )
                flg['position']['exist'] = False
                flg['position']['count'] = 0

        return flg

def check_order(flg):
    # 注文状況確認、約定済みなら以下を実行
    flg['order']['exist'] = False
    flg['order']['count'] = 0
    flg['position']['exist'] = True
    flg['position']['side'] = flg['order']['side']
    flg['position']['price'] = flg['order']['price']
    # 一定時間約定してなければキャンセル

    return flg

# パフォーマンスの記録
def records(data, flg):
    entry_price = flg['position']['price']
    exit_price = round(data['close_price'] * lot)
    trade_cost = round(exit_price * slippage)

    log = 'スリッページ・手数料として ' + str(trade_cost) + '円を考慮します\n'
    flg['records']['log'].append(log)
    flg['records']['slippage'].append(trade_cost)

    # 値幅
    buy_profit = exit_price - entry_price - trade_cost
    sell_profit = -(exit_price - entry_price) - trade_cost

    # 保有時間の記録

    # 利益の有無
    if flg['position']['side'] == 'BUY':
        flg['records']['buy_count'] += 1
        flg['records']['buy_profit'].append(buy_profit)
        flg['records']['buy_return'].append(round(buy_profit / entry_price * 100, 4))
        flg['records']['buy_holding_periods'].append(flg['position']['count'])
        if buy_profit  > 0:
            flg['records']['buy_winning'] += 1
            log = str(buy_profit) + '円の利益です\n'
            flg['records']['log'].append(log)
        else:
        	log = str(buy_profit) + '円の損失です\n'
            flg['records']['log'].append(log)

    if flg['position']['side'] == 'SELL':
        flg['records']['sell_count'] += 1
        flg['records']['sell_profit'].append(sell_profit)
        flg['records']['sell_return'].append(round(sell_profit / entry_price * 100, 4))
        flg['records']['sell_holding_periods'].append(flg['position']['count'])
        if sell_profit > 0:
            flg['records']['sell_winning'] += 1
            log = str(sell_profit) + '円の利益です\n'
            flg['records']['log'].append(log)
        else:
            log = str(sell_profit) + '円の損失です\n'
            flg['records']['log'].append(log)
	    
        return flg

# 集計用の関数
def backtest(flg):
    buy_gross_profit = np.sum(flg['records']['buy_profit'])
    sell_gross_profit = np.sum(flg['records']['sell_profit'])

    print('バックテストの結果')
	print('--------------------------')
	print('買いエントリの成績')
	print('--------------------------')
	print('トレード回数  :  {}回'.format(flg['records']['buy-count']))
	print('勝率         :  {}％'.format(round(flg['records']['buy-winning'] / flg['records']['buy-count'] * 100, 1)))
	print('平均リターン  :  {}％'.format(round(np.average(flg['records']['buy-return']), 4)))
	print('総損益       :  {}円'.format(np.sum(flg['records']['buy-profit'])))
    print('平均保有期間  :  {}足分'.format(round(np.average(flg['records']['buy-holding-periods']),1)))

	
	print('--------------------------')
	print('売りエントリの成績')
	print('--------------------------')
	print('トレード回数  :  {}回'.format(flg['records']['sell-count']))
	print('勝率         :  {}％'.format(round(flg['records']['sell-winning'] / flg['records']['sell-count'] * 100, 1)))
	print('平均リターン  :  {}％'.format(round(np.average(flg['records']['sell-return']), 4)))
	print('総損益       :  {}円'.format(np.sum(flg['records']['sell-profit'])))
	print('平均保有期間  :  {}足分'.format(round(np.average(flg['records']['sell-holding-periods']),1)))
	
	print('--------------------------')
	print('総合の成績')
	print('--------------------------')
	print('総損益       :  {}円'.format(np.sum(flg['records']['sell-profit']) + np.sum(flg['records']['buy-profit'])))
	print('手数料合計    :  {}円'.format(np.sum(flg['records']['slippage'])))
	
	# ログファイルの出力
	info_time = datetime.now().strftime('%Y/%m/%d-%H:%M')
	f = open('./{info_time}-log.txt', 'wt', encoding='UTF-8')
	f.writelines(flg['records']['log'])


In [ ]:

# メインコード
price = get_price_from_file('path')
# price = get_price(chart_sec, after=1514764800)

print('--------------------------')
print('テスト期間:')
print('開始時点: ' + str(price[0]['close_time_dt']))
print('終了時点: ' + str(price[-1]['close_time_dt']))
print(str(len(price)) + '件のローソク足データで検証')
print('--------------------------')

last_data = price[0]

while i < len(price):
	# 未約定を確認
	if flg['order']['exist']:
		flg = check_order(flg)

	data = price[i]
	flg = log_price(data, flg)

	# ポジションがあれば決済条件を確認、条件を満たせば決済
	if flg['position']['exist']:
		flg = close_position(data, last_data, flg)
	else:
		flg = buy_signal(data, last_data, flg)    # buy条件チェック
		flg = sell_signal(data, last_data, flg)   # sell条件チェック
	
	last_data['close_time'] = data['close_time']
	last_data['open_price'] = data['open_price']
	last_data['close_price'] = data['close_price']
	i += 1


backtest(flg)